In [1]:
# !pip install tensorboard==1.15
!nvidia-smi
gpus= "0"

import warnings
warnings.filterwarnings(action='ignore')

import os
os.environ["CUDA_DEVICE_ORDER"]= "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]= gpus;

import torch
gpu_count = torch.cuda.device_count()
print(torch.cuda.is_available())
print(gpu_count)
print(torch.__version__)

Fri Nov 20 13:13:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    Off  | 00000000:05:00.0 Off |                  N/A |
| 53%   69C    P0    60W / 250W |      0MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN X (Pascal)    Off  | 00000000:06:00.0 Off |                  N/A |
| 49%   81C    P2    88W / 250W |   6599MiB / 12196MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

## Setup environment

In [ ]:
# %pip install -q "monai[nibabel, tensorboard]"
!pip install -q "monai[ignite, nibabel, tqdm]" --user

In [ ]:
# !pip install -U git+https://github.com/Project-MONAI/MONAI#egg=MONAI

In [2]:
# # %pip install -q pytorch-lightning==0.9.0
# # %pip uninstall -y pytorch-lightning
# !sudo pip install pytorch-lightning==1.0.3 
# # !pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade
# # psutil
# # !pip install neptune-client --quiet 
# !sudo pip install monai
# # pytorch_lightning.__version__,monai.__version__

## Setup imports

In [3]:
import glob
import os
import shutil
import tempfile

import matplotlib.pyplot as plt
import pytorch_lightning
import torch

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import CacheDataset, list_data_collate
from monai.inferers import sliding_window_inference
from monai.losses import DiceLoss,TverskyLoss,FocalLoss,GeneralizedDiceLoss
from monai.metrics import compute_meandice
from monai.networks.layers import Norm
from monai.networks.nets import *
from monai.transforms import *
from monai.utils import set_determinism

from monai.config import print_config
from monai.data import Dataset, CacheDataset, DataLoader, create_test_image_3d
from monai.engines import EnsembleEvaluator, SupervisedEvaluator, SupervisedTrainer
from monai.handlers import MeanDice, StatsHandler, ValidationHandler
from monai.inferers import SimpleInferer, SlidingWindowInferer
from monai.losses import DiceLoss
from monai.networks.nets import UNet, VNet, HighResNet

print_config()

MONAI version: 0.3.0
Python version: 3.6.8 (default, Jan 14 2019, 11:02:34)  [GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
OS version: Linux (5.4.0-52-generic)
Numpy version: 1.19.4
Pytorch version: 1.7.0
MONAI flags: HAS_EXT = False, USE_COMPILED = False

Optional dependencies:
Pytorch Ignite version: 0.4.2
Nibabel version: 3.2.0
scikit-image version: 0.17.2
Pillow version: 8.0.1
Tensorboard version: 2.3.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.8.1
ITK version: NOT INSTALLED or UNKNOWN VERSION.
tqdm version: 4.51.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [4]:
import glob
path_train = 'Train/'

x_train = sorted(glob.glob(path_train+"*ct*"))
y_train = sorted(glob.glob(path_train+"*seg*"))
path_test = 'Validation/'

x_test = sorted(glob.glob(path_test+"*ct*"))
y_test = sorted(glob.glob(path_test+"*seg*"))

In [5]:
# x_normal = sorted(glob.glob(path_train+"*normal_image*"))
# x_cyst = sorted(glob.glob(path_train+"*cyst_image*"))
# x_cancer = sorted(glob.glob(path_train+"*Cancer_image*"))
# x_SPN = sorted(glob.glob(path_train+"*SPN_image*"))
# x_NET = sorted(glob.glob(path_train+"*NET_image*"))

# y_normal = sorted(glob.glob(path_train+"*normal_seg*"))
# y_cyst = sorted(glob.glob(path_train+"*cyst_seg*"))
# y_cancer = sorted(glob.glob(path_train+"*Cancer_seg*"))
# y_SPN = sorted(glob.glob(path_train+"*SPN_seg*"))
# y_NET = sorted(glob.glob(path_train+"*NET_seg*"))

# print(len(x_normal),len(x_cyst),len(x_cancer),len(x_SPN),len(x_NET))
# print(len(y_normal),len(y_cyst),len(y_cancer),len(y_SPN),len(y_NET))

In [6]:
# from Keeplearning.load_data import *
# import tqdm
# spacings = []
# for idx in tqdm.tqdm(range(len(x_train))):
#     image, origin, spacing = data_load_nii_withSITK(x_train[idx],return_info=True)
#     spacings.append(spacing)

# spacings = np.array(spacings)
# spacings[:,0].mean(),spacings[:,1].mean(),spacings[:,2].mean()
# # (0.40741928562001306, 0.40741928562001306, 4.840921944104015)

# np.median(spacings[:,0]),np.median(spacings[:,1]),np.median(spacings[:,2])
# (.67, .67, 3.0)

In [7]:
import numpy as np
from sklearn.model_selection import KFold

folds = 5
kf = KFold(n_splits=folds,shuffle=True,random_state=0)
kf.get_n_splits(x_train)

train_files = list()
val_files = list()

for train_index, val_index in kf.split(x_train):
    
    FOLDS_train_image = list()
    FOLDS_train_seg = list()
    FOLDS_val_image = list()
    FOLDS_val_seg = list()

    for idx in train_index:
        FOLDS_train_image.append(x_train[idx])
        FOLDS_train_seg.append(y_train[idx])
        
    for idx in val_index:
        FOLDS_val_image.append(x_train[idx])
        FOLDS_val_seg.append(y_train[idx])
    
    train_files.append(
        [
            {"image": img, "label": seg} for img, seg in zip(FOLDS_train_image,FOLDS_train_seg)
        ]
    )

    val_files.append(
        [
             {"image": img, "label": seg} for img, seg in zip(FOLDS_val_image,FOLDS_val_seg)
        ]
    )

test_files = [{"image": img, "label": seg} for img, seg in zip(x_test,y_test)]

In [8]:
# train_files = [{"image": img, "label": seg} for img, seg in zip(x_train,y_train)]
# val_files = [{"image": img, "label": seg} for img, seg in zip(x_valid,y_valid)]
# test_files = [{"image": img, "label": seg} for img, seg in zip(x_test,y_test)]

In [9]:
set_determinism(seed=0)

patch_size = (256,256,16)

train_transforms = Compose(
    [
        LoadNiftid(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),
        RandFlipd(keys=["image", "label"], spatial_axis=[0,1], prob=0.3),
        Spacingd(keys=["image", "label"], pixdim=(.74, .74), mode=("bilinear", "nearest")),
        Spacingd(keys=["image", "label"], pixdim=(.74, .74, 5.0), mode=("nearest", "nearest")),
        RandRotate90d(keys=["image", "label"], prob=0.3, spatial_axes=[0, 1]),
        
        ScaleIntensityRanged(keys=["image"], a_min=-1000.0, a_max=500.0, b_min=0.0, b_max=1.0, clip=True),
#         ScaleIntensityRanged(keys=["image"], a_min=30-500/2, a_max=30+500/2, b_min=0.0, b_max=1.0, clip=True),
#         ScaleIntensityRanged(keys=["label"], a_min=0, a_max=3, b_min=0, b_max=3, clip=True),
        
        RandAdjustContrastd(keys=["image"],gamma=(0.5, 1.5), prob=0.1),
        RandShiftIntensityd(keys=["image"],offsets= (-0.03, 0.03), prob=0.1),
        RandScaleIntensityd(keys=["image"],factors= (0,0.1), prob=0.1),
        RandGaussianSmoothd(keys=["image"],sigma_x=(0, 0.1), sigma_y=(0, 0.11), sigma_z=(0, 0.001), prob=0.1),
        RandHistogramShiftd(keys=["image"],num_control_points=(10,20), prob=0.1),        
        RandAdjustContrastd(keys=["image"],gamma=(0.5, 1.5), prob=0.1),

        RandCropByPosNegLabeld(
            keys=["image", "label"],
            label_key="label",
            spatial_size=patch_size,
            pos=1,
            neg=1,
            num_samples=4,
        ),
#         RandZoomd(
#             keys=["image", "label"],
#             min_zoom=0.9,
#             max_zoom=1.2,
#             mode=("trilinear", "nearest"),
#             align_corners=(True, None),
#             prob=0.16,
#         ),
        CastToTyped(keys=["image", "label"], dtype=(np.float32, np.uint8)),
        ToTensord(keys=["image", "label"]),
    ]
)
val_transforms = Compose(
    [
        LoadNiftid(keys=["image", "label"]),
        AddChanneld(keys=["image", "label"]),              
#         Spacingd(keys=["image", "label"], pixdim=(.67, .67), mode=("bilinear", "nearest")),
#         Spacingd(keys=["image", "label"], pixdim=(.67, .67, 3.0), mode=("nearest", "nearest")),
        Spacingd(keys=["image", "label"], pixdim=(.74, .74), mode=("bilinear", "nearest")),
        Spacingd(keys=["image", "label"], pixdim=(.74, .74, 5.0), mode=("nearest", "nearest")),
        
        ScaleIntensityRanged(keys=["image"], a_min=-1000.0, a_max=500.0, b_min=0.0, b_max=1.0, clip=True),
#         ScaleIntensityRanged(keys=["image"], a_min=30-500/2, a_max=30+500/2, b_min=0.0, b_max=1.0, clip=True),
#         ScaleIntensityRanged(keys=["label"], a_min=0, a_max=3, b_min=0, b_max=3, clip=True),
        CastToTyped(keys=["image", "label"], dtype=(np.float32, np.uint8)),
        ToTensord(keys=["image", "label"]),
    ]
)

In [10]:
fold = 0

set_determinism(seed=0)

train_dss = [Dataset(data=train_files[i], transform=train_transforms) for i in range(folds)]
train_loaders = [DataLoader(train_dss[i], batch_size=4*gpu_count, shuffle=True, num_workers=4) for i in range(folds)]
val_dss = [Dataset(data=val_files[i], transform=val_transforms) for i in range(folds)]
val_loaders = [DataLoader(val_dss[i], batch_size=1, num_workers=4) for i in range(folds)]

train_ds = Dataset(data=train_files[fold], transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=2*gpu_count, shuffle=True, num_workers=2)
val_ds = Dataset(data=val_files[fold], transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=2)
test_ds = Dataset(data=test_files, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=1, num_workers=2)

In [11]:
# class Datasets(nn.Module):
#     def __init__(self,data,transform):
#         self.data = data
#         self.transform = transform
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self,idx):
#         sample = self.data[idx]
#         sample = self.transform(sample)
#         try:
#             sample = sample[0]
#         except:
#             pass
#         if len(torch.unique(sample['seg']))>1:
#             sample['cls'] = torch.tensor([1])
#         else:
#             sample['cls'] = torch.tensor([0])
#         return sample

In [12]:
# # batch = next(iter(test_loader))
# batch = train_ds[0]

# image = batch['image']
# seg = batch['label']

# positive = 0
# negative = 0

# for idx in range(len(seg)):
#     print(idx, torch.unique(seg[idx]))
#     if len(torch.unique(seg[idx]))>1:
#         positive += 1
#     else: 
#         negative += 1
    
# print(positive,negative)

In [13]:
# idx = 0
# image = batch[idx]['image']
# seg = batch[idx]['label']
# print(torch.unique(image),torch.unique(seg))

# for idx in range(image.shape[-1]):
#     plt.figure()
#     plt.title(idx)
#     plt.imshow(image[0,...,idx],cmap='gray')
#     plt.imshow(seg[0,...,idx],alpha=0.3)
# #     print(torch.unique(image[0,...,idx]),torch.unique(seg[0,...,idx]))#,cls)

In [14]:
# from sklearn.metrics import *

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [16]:
class CrossEntropyLoss(nn.Module):
    def __init__(self):
        super().__init__()
#         self.loss = nn.CrossEntropyLoss(weight=torch.tensor([.1,.2,.4,.4]))
        self.loss = nn.CrossEntropyLoss()

    def forward(self, y_pred, y_true):
        # CrossEntropyLoss target needs to have shape (B, D, H, W)
        # Target from pipeline has shape (B, C, D, H, W)
        y_true = torch.squeeze(y_true, dim=1).long()
#         print(y_pred.shape,y_true.shape)
        return self.loss(y_pred, y_true)
    
class DiceCELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.dice = GeneralizedDiceLoss(to_onehot_y=True, softmax=True)
        self.cross_entropy = CrossEntropyLoss()

    def forward(self, y_pred, y_true):
        dice = self.dice(y_pred, y_true)
        cross_entropy = self.cross_entropy(y_pred, y_true)
        return dice + cross_entropy    

In [17]:
# from sklearn.metrics import *
from sklearn.metrics import f1_score

def train():
    model.train()
    epoch_loss = 0
    epoch_metric = list()
    step = 0
    
    for batch_data in tqdm.tqdm(train_loader):
        step += 1
        inputs, labels = batch_data['image'].to(device), batch_data['label'].to(device)
        optimizer.zero_grad()
        if amp and scaler is not None:
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            loss.backward()
            optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / step

def valid():
    model.eval()
    epoch_val_loss = 0
    metric = 0.
    step = 0

    with torch.no_grad():
        for val_data in tqdm.tqdm(val_loader):  
            step += 1              
            inputs, labels = val_data['image'].to(device), val_data['label'].to(device)
            roi_size = patch_size
            sw_batch_size = 2
            if amp:
                with torch.cuda.amp.autocast():
                    outputs = sliding_window_inference(inputs, roi_size, sw_batch_size, model)
            else:
                outputs = sliding_window_inference(inputs, roi_size, sw_batch_size, model)

            loss = loss_function(outputs, labels)
#             value = compute_meandice(y_pred=outputs, y=labels, include_background=False, to_onehot_y=True, mutually_exclusive=True)

            epoch_val_loss += loss.item()
#             metric += value.sum().item()
            
    return epoch_val_loss / step

In [18]:
import tqdm
amp=True
scaler = torch.cuda.amp.GradScaler() if amp else None

epoch_num = 300
lr = 1e-4

# model & gpu setting
device = torch.device('cuda')
model = UNet(dimensions=3, in_channels=1, out_channels=2, channels=(16, 32, 64, 128, 256), strides=(2, 2, 2, 2), num_res_units=2, norm=Norm.BATCH)
# model = VNet(spatial_dims=3, in_channels=1, out_channels=4)
# model = monai.networks.nets.SegResNetVAE(patch_size, vae_estimate_std=True, vae_nz=256, spatial_dims=3, init_filters=8, in_channels=1, out_channels=4, dropout_prob=None, norm_name='group', num_groups=8, use_conv_final=True, blocks_down=(1, 2, 2, 4), blocks_up=(1, 1, 1), upsample_mode='nontrainable')

model_w = torch.load(str(fold)+'best_unet.pth')
model.load_state_dict(model_w)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model= model.to(device)

# loss & optimizer setting
loss_function = DiceCELoss()
# optimizer = torch.optim.Adam(model.parameters(), lr)
optimizer = monai.optimizers.Novograd(model.parameters(), lr*3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,epoch_num)

In [ ]:
# !pip install livelossplot --quiet
import numpy as np
from livelossplot import PlotLosses

groups = {'log-loss': ['train_loss', 'val_loss']}
plotlosses = PlotLosses(groups=groups)

epoch_loss_values = list()
epoch_val_loss_values = list()
epoch_metric_values = list()

for epoch in range(epoch_num):
    print('fold       {} / {}  '.format(fold,folds))
    print('epoch     ',epoch)
    print('train     ', epoch_loss_values)
    print('validation', epoch_val_loss_values)
    epoch_loss = train()
    epoch_loss_values.append(epoch_loss)
    
    epoch_val_loss = valid()
    epoch_val_loss_values.append(epoch_val_loss)
    
    if epoch >= 3 and epoch_val_loss == np.min(epoch_val_loss_values):
        if torch.cuda.device_count() > 1:
            torch.save(model.module.state_dict(),str(fold)+'best_unet.pth')
        else:
            torch.save(model.state_dict(), str(fold)+'best_unet.pth')
        print('saved new best metric model')
    
    plotlosses.update({
    'train_loss': epoch_loss,
    'val_loss': epoch_val_loss
    })
    plotlosses.send()

In [ ]:
print(str(fold)+'best_unet.pth')
if torch.cuda.device_count() > 1:
    torch.save(model.module.state_dict(),str(fold)+'best_unet.pth')
else:
    torch.save(model.state_dict(), str(fold)+'best_unet.pth')

In [ ]:
# pr = torch.rand(2,4,64,64,64)
# gt = torch.randint(0,4,(2,64,64,64))

# pr = torch.log_softmax(pr, dim=1)
# temp = torch.argmax(pr, dim =1)

# for idx in range(2):
#     pr_ = temp[idx]
#     gt_ = gt[idx]

#     pr_[pr_==2] = 0
#     gt_[gt_==2] = 0
#     gt_[gt_==3] = 0
#     print(pr.shape,torch.unique(pr_),gt.shape,torch.unique(gt_))

#     dice = f1_score(gt_.flatten(),pr_.flatten(),average=None)
#     print(dice)

# def metric_f1(pr,gt):
#     """
#     pr : shape (N,C,H,W,(D))
#     gt : shape (N,H,W,(D))
#     """
    
#     N = pr.shape[0]
#     C = pr.shape[1]
#     assert C > 1
    
#     pr = torch.log_softmax(pr, dim=1)
#     pr = torch.argmax(pr, dim=1)
    
#     result = torch.zeros(N,C)
    
#     for idx in range(N):
#         pr_ = pr[idx]
#         gt_ = gt[idx]
        
#         pr_[pr_==2] = 0
#         gt_[gt_==2] = 0
#         gt_[gt_==3] = 0
#         print(pr.shape,torch.unique(pr_),gt.shape,torch.unique(gt_))
  
# #         dice = f1_score(gt_.cpu().detach().numpy().flatten(),pr_.cpu().detach().numpy().flatten(),average=None)
#         dice = f1_score(gt_.flatten(),pr_.flatten(),average=None)
#         print(dice)
#     return result

# metric_f1(pr,gt)

In [19]:
model_w = torch.load(str(fold)+'best_unet.pth')
model.load_state_dict(model_w)

<All keys matched successfully>

In [20]:
model.eval()
with torch.no_grad():
    for i, val_data in enumerate(val_loader):
        image = val_data["image"] 
        label = val_data["label"]
        fname_image = val_data["image_meta_dict"]['filename_or_obj'][0]
        fname_label = val_data["label_meta_dict"]['filename_or_obj'][0]
        affine = val_data["image_meta_dict"]['affine'][0]
        original_affine = val_data["image_meta_dict"]['original_affine'][0]
        spatial_shape = val_data["image_meta_dict"]['spatial_shape'][0]
        
        roi_size = patch_size
        sw_batch_size = 4
        outputs = sliding_window_inference(val_data["image"].to(device), roi_size, sw_batch_size, model, 0.5, 'gaussian')
        outputs = torch.argmax(outputs, dim=1).unsqueeze(1)
        print(label.shape,torch.unique(label),torch.unique(outputs))
        
#         # for color visualization
#         label_ = label.clone()
#         outputs_ = outputs.clone()
#         for idx in range(2):
#             label_[0,0,0,idx,:] = idx
#             outputs_[0,0,0,idx,:] = idx
        
#         for idx in range(label.shape[-1]):
#             if len(torch.unique(label[...,idx]))>=2:
#                 plt.figure("check", (18, 6))
#                 plt.subplot(1, 3, 1)
#                 plt.title(f"{fname_image} {idx}")
#                 plt.imshow(image[0, 0, :, :, idx], cmap="gray")
#                 plt.subplot(1, 3, 2)
#                 plt.title(f"label {i}")
#                 plt.imshow(image[0, 0, :, :, idx], cmap="gray")
#                 plt.imshow(label_[0, 0, :, :, idx], alpha=0.5)
#                 plt.subplot(1, 3, 3)
#                 plt.title(f"output {i}")
#                 plt.imshow(image[0, 0, :, :, idx], cmap="gray")
#                 plt.imshow(outputs_.detach().cpu()[0, 0,:, :,idx], alpha=0.5)
#                 plt.show()                
                
        monai.data.write_nifti(data=image.cpu().detach().numpy().squeeze(),
                               file_name='Output/'+str(fold)+'/'+fname_image.split('/')[-1],
                               resample=True,
                               affine=affine, 
                               mode="bilinear",
                               target_affine=original_affine,
                               output_spatial_shape=spatial_shape)
        
        monai.data.write_nifti(data=outputs.cpu().detach().numpy().squeeze(),
                               file_name='Output/'+str(fold)+'/'+fname_label.split('/')[-1],
                               resample=True,
                               affine=affine,
                               mode="nearest",
                               target_affine=original_affine,
                               output_spatial_shape=spatial_shape)

torch.Size([1, 1, 513, 513, 73]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 585, 585, 61]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 638, 638, 54]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 513, 513, 73]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 651, 651, 55]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 599, 599, 77]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 520, 520, 63]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 472, 472, 58]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 512, 512, 53]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size([1, 1, 617, 617, 73]) tensor([0, 1], dtype=torch.uint8) tensor([0, 1], device='cuda:0')
torch.Size

In [ ]:
models= [model,model,model]

In [ ]:
# def ensemble_evaluate(post_transforms, models):
#     evaluator = EnsembleEvaluator(
#         device=device,
#         val_data_loader=test_loader,
#         pred_keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
#         networks=models,
#         inferer=SlidingWindowInferer(roi_size=patch_size, sw_batch_size=4, overlap=0.5),
#         post_transform=post_transforms,
#         key_val_metric={
#             "test_mean_dice": MeanDice(
#                 include_background=True, to_onehot_y=True, output_transform=lambda x: (x["pred"], x["label"]),
#             )
#         },
#     )
#     evaluator.run()
    
# mean_post_transforms = Compose(
#     [
#         MeanEnsembled(
#             keys=["pred0", "pred1", "pred2", "pred3", "pred4"],
#             output_key="pred",
#             # in this particular example, we use validation metrics as weights
#             weights=[0.95, 0.94, 0.95, 0.94, 0.90],
#         ),
#         Activationsd(keys="pred", sigmoid=True),
#         AsDiscreted(keys="pred", threshold_values=True),
#     ]
# )
# ensemble_evaluate(mean_post_transforms, models)

In [ ]:
from ignite.contrib.handlers import ProgressBar
val_handlers = [
    ProgressBar(),
]

def ensemble_evaluate(post_transforms, models):
    evaluator = EnsembleEvaluator(
        device=device,
        val_data_loader=test_loader,
        pred_keys=["pred0", "pred1", "pred2"],
        networks=models,
        val_handlers=val_handlers,
        inferer=SlidingWindowInferer(roi_size=patch_size, sw_batch_size=4, overlap=0.5),
        post_transform=post_transforms,
#         key_val_metric={
#             "test_mean_dice": MeanDice(nclude_background=True, to_onehot_y=True, output_transform=lambda x: (x["pred"], x["label"]),
#             )
#         },
    )
    evaluator.run()
    
# mean_post_transforms = Compose(
#     [
#         MeanEnsembled(
#             keys=["pred0", "pred1", "pred2"],
#             output_key="pred",
#             weights=[0.95, 0.94, 0.95],
#         ),
#         Activationsd(keys="pred", softmax=True),
#     ]
# )

mean_post_transforms = Compose(
    [
        VoteEnsembled(
            keys=["pred0", "pred1", "pred2"],
            output_key="pred",
        ),
        Activationsd(keys="pred", softmax=True),
    ]
)
pred = ensemble_evaluate(mean_post_transforms, models)